In [26]:
from sklearn.datasets import load_iris
from sklearn.model_selection import train_test_split ,GridSearchCV, RandomizedSearchCV
import pandas as pd
from sklearn.metrics import accuracy_score

In [12]:
data = load_iris()
x  = pd.DataFrame(data.data , columns = data.feature_names)
y = data.target


In [13]:

x_train , x_test , y_train , y_test = train_test_split(x , y, test_size=0.2)

In [18]:
x_train.shape

(120, 4)

In [15]:
x_train.head()

,sepal length (cm),sepal width (cm),petal length (cm),petal width (cm)
113,5.7,2.5,5.0,2.0
20,5.4,3.4,1.7,0.2
102,7.1,3.0,5.9,2.1
126,6.2,2.8,4.8,1.8
42,4.4,3.2,1.3,0.2


In [14]:
y_train[:5]

array([2, 0, 2, 2, 0])

In [19]:
from sklearn.ensemble import RandomForestClassifier
model = RandomForestClassifier()

In [28]:
model.fit(x_train , y_train)
preds = model.predict(x_test)
print("accuracy  : " , accuracy_score(preds,y_test))

accuracy  :  0.9


In [21]:
model.get_params()

{'bootstrap': True,
 'ccp_alpha': 0.0,
 'class_weight': None,
 'criterion': 'gini',
 'max_depth': None,
 'max_features': 'sqrt',
 'max_leaf_nodes': None,
 'max_samples': None,
 'min_impurity_decrease': 0.0,
 'min_samples_leaf': 1,
 'min_samples_split': 2,
 'min_weight_fraction_leaf': 0.0,
 'n_estimators': 100,
 'n_jobs': None,
 'oob_score': False,
 'random_state': None,
 'verbose': 0,
 'warm_start': False}

In [23]:
grid = {
    'n_estimators' : [10,20,30,40,60,80,100],
    'criterion' : ['gini' , 'entropy'],
    'max_depth' : [None , 10 , 20],
    'min_samples_split' : [2,4,6],
    'min_samples_leaf' : [1,2,4]
}

In [32]:
gridSearch = GridSearchCV(estimator=model , param_grid=grid, cv=5)
gridSearch.fit(x_train , y_train)

GridSearchCV(cv=5, estimator=RandomForestClassifier(),
             param_grid={'criterion': ['gini', 'entropy'],
                         'max_depth': [None, 10, 20],
                         'min_samples_leaf': [1, 2, 4],
                         'min_samples_split': [2, 4, 6],
                         'n_estimators': [10, 20, 30, 40, 60, 80, 100]})

In [33]:
print(gridSearch.best_params_)

{'criterion': 'gini', 'max_depth': None, 'min_samples_leaf': 4, 'min_samples_split': 4, 'n_estimators': 10}


In [36]:
print("tuned acc : " ,gridSearch.best_score_)

tuned acc :  0.975


In [38]:
randomSearch = RandomizedSearchCV(estimator=model , param_distributions=grid , cv=5 , n_iter=10)

In [46]:
from sklearn.svm import SVC

classifier = SVC()
classifier.get_params()


{'C': 1.0,
 'break_ties': False,
 'cache_size': 200,
 'class_weight': None,
 'coef0': 0.0,
 'decision_function_shape': 'ovr',
 'degree': 3,
 'gamma': 'scale',
 'kernel': 'rbf',
 'max_iter': -1,
 'probability': False,
 'random_state': None,
 'shrinking': True,
 'tol': 0.001,
 'verbose': False}

In [49]:
svm_grid  = {
    'C' : [0.5 , 1.0 , 1.5 , 2.0 ],
    'degree' : [2,3,4,5]

}

In [51]:
svmSearch = GridSearchCV(estimator=classifier , param_grid=svm_grid , cv=5 )
svmSearch.fit(x_train,y_train)
svmSearch.best_params_
svmSearch.best_score_

0.9583333333333334

KERAS TUNING

In [54]:
import tensorflow 
from tensorflow import keras
from keras_tuner.tuners import RandomSearch

In [55]:
def build_model(hp):
    model = keras.Sequential()
    model.add(keras.layers.Dense(units=hp.Int('units',
                                              min_value=32,
                                              max_value=512,
                                              step=32),
                                              activation='relu',
                                              input_shape=(4,)))
    model.add(keras.layers.Dense(units=3 , activation='softmax'))

    model.compile(optimizer=keras.optimizers.Adam(hp.Choice('learning_rate' , values=[1e-2, 1e-3 , 1e-4])),
                  loss='sparse_categorical_crossentropy', 
                  metrics=['accuracy'])

    return model

In [56]:
tuner = RandomSearch(build_model, objective='val_accuracy', max_trials=5)

In [57]:
tuner.search(x_train,y_train ,epochs=5, validation_data=(x_test,y_test))

Trial 5 Complete [00h 00m 01s]
val_accuracy: 0.9333333373069763

Best val_accuracy So Far: 0.9333333373069763
Total elapsed time: 00h 00m 07s


In [58]:
best_hp = tuner.get_best_hyperparameters(num_trials=1)[0]

In [61]:
model = tuner.hypermodel.build(best_hp)
model.fit(x_train,y_train , epochs=10)

Epoch 1/10
4/4 [==============================] - 0s 4ms/step - loss: 1.0232 - accuracy: 0.5000
Epoch 2/10
4/4 [==============================] - 0s 2ms/step - loss: 0.9404 - accuracy: 0.5417
Epoch 3/10
4/4 [==============================] - 0s 1ms/step - loss: 0.5767 - accuracy: 0.7333
Epoch 4/10
4/4 [==============================] - 0s 3ms/step - loss: 0.5862 - accuracy: 0.6917
Epoch 5/10
4/4 [==============================] - 0s 2ms/step - loss: 0.4547 - accuracy: 0.7750
Epoch 6/10
4/4 [==============================] - 0s 2ms/step - loss: 0.4135 - accuracy: 0.8000
Epoch 7/10
4/4 [==============================] - 0s 4ms/step - loss: 0.3787 - accuracy: 0.7167
Epoch 8/10
4/4 [==============================] - 0s 2ms/step - loss: 0.3105 - accuracy: 0.9250
Epoch 9/10
4/4 [==============================] - 0s 5ms/step - loss: 0.2785 - accuracy: 0.9417
Epoch 10/10
4/4 [==============================] - 0s 4ms/step - loss: 0.2602 - accuracy: 0.9333


In [64]:
print("accuracy" ,model.evaluate(x_test,y_test)[1])

1/1 [==============================] - 0s 79ms/step - loss: 0.2995 - accuracy: 0.9667
accuracy 0.9666666388511658


Hyper-Parameter on Fashion Mnist

In [65]:
from keras.datasets import fashion_mnist

(train_data,train_label) , (test_data,test_label) = fashion_mnist.load_data()

In [67]:
train_data.shape

(60000, 28, 28)

In [68]:
train_data = train_data/255.0
test_data = test_data/255.0

In [85]:
paramGrid = {
    "learning" : [1e-1 , 1e-2 , 1e-3],
    "neurons" : [32,64,128]
}

In [71]:
def create_model(learning , neurons):
    model = keras.Sequential()
    model.add(keras.layers.Flatten(input_shape=(28,28)))
    model.add(keras.layers.Dense(neurons ,activation='relu'))
    model.add(keras.layers.Dense(10, activation='softmax'))
    model.compile( 'learning_rate' , values=learning ,optimizer=keras.optimizers.Adam ,loss= "sparse_categorical_crossentropy", metrics="accuracy" )
    return model

In [86]:
random_search = RandomizedSearchCV(
    estimator=build_model,
    param_distributions=paramGrid,
    n_iter=10,  # Number of random hyperparameter combinations to try
    cv=3,  # Number of folds for cross-validation
)
